# Model M61

External economies of scale

|Markets  |    X  |      Y  |      W   |       CONS|
|---------|-------|---------|----------|------------|
|    PX   |  100  |         |  -100    |            |     
|    PY   |       |   100   |  -100    |            |        
|    PU   |       |         |   200    |       -200 |         
|    PW   |  -40  |   -60   |          |        100 |        
|    PZ   |  -60  |   -40   |          |        100 |       

In [29]:
using MPSGE_MP

In [30]:
M61 = MPSGEModel()

@parameters(M61, begin
    ENDOW, 1, (description = "Size index for the economy",)
    β, .2, (description = "External economies parameter",)
end)


@sectors(M61, begin
	X,	(description = "Activity level for sector X",)
	Y,	(description = "Activity level for sector Y",)
	W,	(description = "Activity level for sector W (Hicksian welfare index)",)
end)
@commodities(M61, begin
	PX,	(description = "Price index for commodity X",)
	PY,	(description = "Price index for commodity Y",)
	PW,	(description = "Price index for primary factor L",)
	PZ,	(description = "Price index for primary factor S",)
	PU,	(description = "Price index for welfare (expenditure function)",)
end)
@consumers(M61, begin
	CONS,	(description = "Income level for consumer CONS",)
end)
@auxiliaries(M61, begin
	XQADJ,	(description = "Quantity adjustment   (positive when X>1)",)
	XPADJ,	(description = "X output subsidy rate (positive when X>1)",)
end)

@production(M61, X, [s=1,t=0], begin
    @output(PX, 100, t, taxes = [Tax(CONS, XPADJ*-1)])
    @input(PW, 40, s)
    @input(PZ, 60, s)
end)

@production(M61, Y, [s=1,t=0], begin
    @output(PY, 100, t)
    @input(PW, 60, s)
    @input(PZ, 40, s)
end)


@production(M61, W, [s=1,t=0], begin
    @output(PU, 200, t)
    @input(PX, 100, s)
    @input(PY, 100, s)
end)

@demand(M61, CONS, begin
    @final_demand(PU, 200)
end,begin
    @endowment(PW, ENDOW*100)
    @endowment(PZ, ENDOW*100)
    @endowment(PX, 100*XQADJ)
end)

@aux_constraint(M61, XQADJ, 
    XQADJ - (X^(1/(1-β)) - X)
)

@aux_constraint(M61, XPADJ, 
    XPADJ*X - XQADJ
);

# Benchmark

In [31]:
using JuMP

JuMP.delete_lower_bound(get_variable(XQADJ))
JuMP.delete_lower_bound(get_variable(XPADJ))

In [32]:
solve!(M61, cumulative_iteration_limit = 0)
generate_report(M61)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_2997.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 10

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Default price normalization using income for CONS - This value is fixed. Unfix with unfix(CONS).

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.0,0.0
2,Y,1.0,0.0
3,W,1.0,0.0
4,PU,1.0,0.0
5,PW,1.0,0.0
6,PZ,1.0,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,CONS,200.0,0.0


# Counterfactual

In [33]:
set_value!(ENDOW, 2)

solve!(M61)
generate_report(M61)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_29C7.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.8144e+02             0.0e+00 (market_clearance[PW)
    1     2     0    10 6.0032e+01  8.0e-01    0.0e+00 (market_clearance[PW)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     3     2 6.0032e+01           I 0.0e+00 4.0e+01 (market_clearanc)
    1     1     4     3 1.1357e+01  1.0e+00 SM 0.0e+00 1.1e+01 (market_clearanc)
    2     1     5     4 5.2786e-01  1.0e+00 SO 0.0e+00 4.1e-01 (market_clearanc)
    3     1     6     5 4.3223e-01  1.0e+00 SO 0.0e+00 4.3e-01 (market_clearanc)
    4     1     7     6 4.4939e-02  1.0e+0

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,2.0,-7.10543e-15
2,Y,2.0,1.06581e-14
3,W,2.18102,1.42109e-14
4,PU,0.458502,1.13687e-13
5,PW,0.5,2.84217e-14
6,PZ,0.5,-2.84217e-14
7,PY,0.5,5.68434e-14
8,PX,0.420448,1.42109e-14
9,CONS,200.0,0.0


# Counterfactual

In [34]:
set_value!(ENDOW, .8)

solve!(M61)
generate_report(M61)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_2A16.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.3789e+00             0.0e+00 (market_clearance[PW)
pn_search terminated: no progress.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    13     1 1.3789e+00           I 0.0e+00 9.8e-01 (market_clearanc)
    1     1    14     2 1.8566e+00  1.0e+00 SO 0.0e+00 1.2e+00 (market_clearanc)
    2     1    15     3 3.0202e-01  1.0e+00 SO 0.0e+00 3.0e-01 (market_clearanc)
    3     1    16     4 7.6043e-04  1.0e+00 SO 0.0e+00 7.6e-04 (market_clearanc)
    4     1    17     5 3.6705e-09  1.0e+00 SO 0.0e+00 3.7e-09 (market_clearanc)

Major Iterations. . . . 4
Minor Iteration

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.8,-3.70681e-11
2,Y,0.8,-1.10134e-12
3,W,0.777994,-3.97904e-13
4,PU,1.28536,3.66947e-9
5,PW,1.25,-5.05764e-11
6,PZ,1.25,5.19265e-11
7,PY,1.25,2.251e-11
8,PX,1.32171,-2.10463e-11
9,CONS,200.0,4.74952e-9
